# Git

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%%bash
# run below in terminal
eval `ssh-agent -s`
ssh-add /content/drive/MyDrive/github/colab
ssh -T git@github.com -y
git config --global user.email "haogeh@andrew.cmu.edu"
git config --global user.name "haogeh"

Agent pid 745


Identity added: /content/drive/MyDrive/github/colab (haogeh@andrew.cmu.edu)


In [5]:
!cd /content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare

# Setup

In [6]:
pip install openai==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [7]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [2]:
import openai
from openai import OpenAI

print(openai.__version__) # version - 1.12.0


1.12.0


### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

In [3]:
cd /content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare

/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare


In [4]:
ls

credentials.json  data_3.json  examples/         llm.log    starter_code.ipynb
data_2.json       data.json    instructions.txt  README.md


# Load data and credentials

In [5]:
import json

# Update the path if your file is in a different location
file_path = 'data.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)

# To verify, print a part of the data
print(patient_data['patient_demographics'])



{'name': 'John Doe', 'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}


In [6]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.

credentials_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']
umls_api_key = credentials['umls_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


# Explore data

In [7]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [8]:
print(f'Keys in data: {(patient_data.keys())}')

Keys in data: dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])


In [9]:
# Look at encounters data, there are multiple
patient_data['encounters']

[{'date': '2024-02-10',
  'type': 'Admission',
  'reason': 'Cough, shortness of breath, hemoptysis, fever'},
 {'date': '2024-02-14', 'type': 'Discharge'}]

In [11]:
def del_sensitive_data(data,sensitive_key = ['name','firstname','lastname','patient_id']):
  keys = list(data.keys())
  for key in keys:
    if type(data[key]) is dict:
      del_sensitive_data(data[key])
    else:
      if key.lower() in sensitive_key:
        del data[key]

In [12]:
llm_patient_data = patient_data.copy()
del_sensitive_data(llm_patient_data)
llm_patient_data

{'patient_demographics': {'age': 70,
  'gender': 'Male',
  'admission_date': '2024-02-10',
  'discharge_date': '2024-02-14'},
 'diagnoses': [{'date': '2024-02-10',
   'diagnosis_code': 'J18.1',
   'description': 'Lobar pneumonia, unspecified organism'}],
 'drg': {'code': '193',
  'description': 'Simple pneumonia and pleurisy with MCC'},
 'encounters': [{'date': '2024-02-10',
   'type': 'Admission',
   'reason': 'Cough, shortness of breath, hemoptysis, fever'},
  {'date': '2024-02-14', 'type': 'Discharge'}],
 'flowsheets': [{'date': '2024-02-10',
   'time': '08:00',
   'temperature': '38.5°C',
   'heart_rate': '90 bpm',
   'blood_pressure': '130/85 mmHg',
   'respiratory_rate': '20 breaths/min',
   'oxygen_saturation': '92%'},
  {'date': '2024-02-11',
   'time': '08:00',
   'temperature': '37.8°C',
   'heart_rate': '85 bpm',
   'blood_pressure': '125/80 mmHg',
   'respiratory_rate': '18 breaths/min',
   'oxygen_saturation': '94%'},
  {'date': '2024-02-12',
   'time': '08:00',
   'temper

# Logger

In [13]:
import logging

#Setup the logger
logger = logging.getLogger('logger')  # Create a logger object
logger.setLevel(logging.DEBUG)  # Set the minimum logging level

#Create a file handler that logs even debug messages
file_handler = logging.FileHandler('llm.log')
file_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)


# Chain-of-thought solution


In [14]:
client = OpenAI(api_key=openai_api_key)

In [15]:
def ok_for_discharge(patient_data):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =  {
                "role": "user",
                "content": f"""w
                              You are a physician deciding whether to discharge a patient. Please decide whether the given patient is safe to discharge.
                              Consider all information provided, particulary the doctors' notes.
                              Write the output in this format: Yes/No
                              Data: {patient_data}
                """,
            }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [16]:
def generate_patient_summary(patient_data, additional_prompts='', instructions='',example=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Any additional prompts

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =  {
                "role": "user",
                "content": f"""
                              You are a physician writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.
                              {additional_prompts}
                              Write the output in this format: {instructions}
                              Example: {example}
                              Data: {patient_data}
                """,
            }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [18]:
#Instructions for a good discharge summary :

# Define the path to your file
inst_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/instructions.txt'
eg_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/examples/eg1.txt'


with open(inst_path, 'r') as file:
    instructions = file.read()
    print(instructions)

with open(eg_path, 'r') as file:
    example = file.read()
    print(example)

Step 1
Patient Identification: Begin by filling in the patient's essential information, like age, sex, and date of birth.

Step 2
Admission and Discharge Dates: Include the dates the patient was admitted and discharged.

Step 3
Reason for Admission: Detail the primary diagnosis or condition that necessitated hospitalization.

Step 4 
Past Medical History:Important concurrent diagnoses (e.g., CHF with specific ejection fraction, history of myocardial infarctions, significant lesions, HIV status)

Step 5
Procedures Performed: Describe any surgical or non-surgical procedures done during the hospital stay.

Step 6
Medications Prescribed: List the medications prescribed to the patient at discharge, including dosages and frequency.

Step 7
Patient's Condition at Discharge: Provide a summary of the patient's health status at discharge.

Step 8
Follow-up Care Instructions: Offer detailed instructions for post-hospital care, including medications, dietary restrictions, physical activity recomme

In [19]:
print(ok_for_discharge(llm_patient_data))

INFO:logger:Q: {'role': 'user', 'content': 'w\n                              You are a physician deciding whether to discharge a patient. Please decide whether the given patient is safe to discharge.\n                              Consider all information provided, particulary the doctors\' notes.\n                              Write the output in this format: Yes/No\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneumonia and pleurisy with MCC\'}, \'encounters\': [{\'date\': \'2024-02-10\', \'type\': \'Admission\', \'reason\': \'Cough, shortness of breath, hemoptysis, fever\'}, {\'date\': \'2024-02-14\', \'type\': \'Discharge\'}], \'flowsheets\': [{\'date\': \'2024-02-10\',

Yes


In [33]:
additional_prompts='Focus on precision and minimize any assumptions or hallucinations not directly supported by the data. Do not display the step number in the output.'
print(generate_patient_summary(llm_patient_data,additional_prompts,instructions))

INFO:logger:Q: {'role': 'user', 'content': '\n                              You are a physician writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.\n                              Focus on precision and minimize any assumptions or hallucinations not directly supported by the data. Do not display the step number in the output.\n                              Write the output in this format: Step 1\nPatient Identification: Begin by filling in the patient\'s essential information, like age, sex, and date of birth.\n\nStep 2\nAdmission and Discharge Dates: Include the dates the patient was admitted and discharged.\n\nStep 3\nReason for Admission: Detail the primary diagnosis or condition that necessitated hospitalization.\n\nStep 4 \nPast Medical History:Important concurrent diagnoses (e.g., CHF with specific ejection fraction, history of myocardial infarctions, significant lesions, HIV status)\n\nStep 5\nProcedures Performed: Describe any s

Patient Identification: 
- Age: 70
- Gender: Male

Admission and Discharge Dates: 
- Admission Date: 2024-02-10
- Discharge Date: 2024-02-14

Reason for Admission: 
- Diagnosis: Lobar pneumonia, unspecified organism

Past Medical History: 
- No specific information provided

Procedures Performed: 
- No specific procedures mentioned

Medications Prescribed at Discharge: 
- Amoxicillin PO 250 mg daily
- Atorvastatin PO 20 mg nightly

Patient's Condition at Discharge: 
- Improved and medically fit for discharge

Follow-up Care Instructions:
- Take oral antibiotics for 5 more days
- Follow-up in outpatient clinic in two weeks

Hospital Stay Summary: 
- Admitted with symptoms indicative of pneumonia
- Responded well to empirical antibiotic therapy
- Signs of improvement noted throughout the hospital stay
- Temperature normalized, oxygen saturation stable
- Continued improvement in CRP and WBC levels
- Discharged with oral antibiotics for further treatment.


# RAG

In [21]:
def generate_rag_query(patient_data):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message = {
                "role": "user",
                "content": f"""
                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.
                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.
                              Based on the patient data, generate a query for the UMLS.
                              Write the output in this format: one word for the disease
                              Example: Diabete
                              Data: {patient_data}
                """,
    }

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [22]:
import requests

def search_umls(term, api_key):
    search_url = f"https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        'apiKey': api_key,
        'string':f'{term}'
    }
    response = requests.get(search_url, params=params)
    logger.info(f'Search UMLS: term: {term}')
    return response.json()

api_key = umls_api_key

In [23]:
def get_concept_definition(cui, api_key):
    """Get the concept definition using the CUI and service ticket."""
    api_url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/definitions"
    params = {
        'apiKey': api_key,
    }
    response = requests.get(api_url, params=params)
    logger.info(f'''
    Get Concept Definition: cui: {cui}
    Response: {response.json()}
                ''')
    return response.json()

In [24]:
def generate_rag_explanation(patient_data,umls):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =   {
                "role": "user",
                "content": f"""
                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.
                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.
                              Based on the patient data and retrieved UMLS results, generate a brief explanation (100-200 words) of the disease for the patient.
                              Do not use unnecessary medical terminology to ensure the patient can understand the disease.
                              Data: {patient_data}
                              UMLS result: {umls}
                              Format:
                                Terminology:
                                Explanation:
                """,
    }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [25]:
query = generate_rag_query(llm_patient_data)
print(query)

INFO:logger:Q: {'role': 'user', 'content': '\n                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.\n                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.\n                              Based on the patient data, generate a query for the UMLS.\n                              Write the output in this format: one word for the disease\n                              Example: Diabete\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneumonia and pleurisy with MCC\'}, \'encounters\': [{\'date\': \'2

Query: lobar pneumonia


In [26]:
results = search_umls(query, api_key)
print(results)
cui = None
if results['result']['results']:
  cui = results['result']['results'][0]['ui']
print(cui)

INFO:logger:Search UMLS: term: Query: lobar pneumonia


{'pageSize': 25, 'pageNumber': 1, 'result': {'classType': 'searchResults', 'results': [], 'recCount': 0}, 'partialSearch': 'https://uts-ws.nlm.nih.gov/rest/search/current?string=Query%3A%20lobar%20pneumonia&partialSearch=true'}
None


In [27]:
if cui:
  results = get_concept_definition(cui, api_key)
  print(results)
definition = results['result']

In [28]:
print(generate_rag_explanation(llm_patient_data,definition))

INFO:logger:Q: {'role': 'user', 'content': '\n                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.\n                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.\n                              Based on the patient data and retrieved UMLS results, generate a brief explanation (100-200 words) of the disease for the patient.\n                              Do not use unnecessary medical terminology to ensure the patient can understand the disease.\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneumon

Terminology: Lobar pneumonia

Explanation: Lobar pneumonia is a type of lung infection that primarily affects one or more lobes of the lungs. It is often caused by bacteria and can lead to symptoms such as cough, shortness of breath, fever, and chest pain. In this case, the patient was diagnosed with lobar pneumonia with unspecified organism causing the infection. Treatment typically involves antibiotics to help clear the infection and supportive care to alleviate symptoms. With proper medical management, patients can recover well from lobar pneumonia. It is important to follow the prescribed antibiotics regimen and attend follow-up appointments as advised by your healthcare provider to ensure a full recovery.


## Close Logger

In [29]:
file_handler.close()
logger.removeHandler(file_handler)